In [1]:
# encoding: UTF-8
"""
展示如何执行策略回测。
"""
from vnpy.trader.app.ctaStrategy import BacktestingEngine
import pandas as pd
from vnpy.trader.utils import htmlplot
import json
import logging

if __name__ == '__main__':
    from Strategy_Boll_arrayPerpear import StrategyBoll
    # 创建回测引擎
    engine = BacktestingEngine()
    logging.basicConfig(level=logging.ERROR)

    # 设置引擎的回测模式为K线
    engine.setBacktestingMode(engine.BAR_MODE)
    # 设置使用的历史数据库
    engine.setDatabase('VnTrader_1Min_Db')

    # 设置回测用的数据起始日期，initHours 默认值为 0
    engine.setStartDate('20180610 06:00:00',initHours=120)   
    engine.setEndDate('20190517 06:00:00')
    engine.setDB_URI('mongodb://192.168.0.104:27017')
    # 设置产品相关参数
    engine.setCapital(1000)  # 设置起始资金，默认值是1,000,000
    contracts = [
        {
        "symbol":'eth.usd.q:okef',
        "size" : 10,
        "priceTick" : 0.001,
        "rate" : 5/10000,
        "slippage" : 0.001
        },
        {
        "symbol":'btc.usd.q:okef',
        "size" : 10,
        "priceTick" : 0.001,
        "rate" : 5/10000,
        "slippage" : 0.001
        }
        ]
    engine.setContracts(contracts)     # 设置回测合约相关数据
    engine.setLog(True, "./log") 
     # engine.setCachePath("C:\\Users\\Administrator\\Desktop\\回测用文件\\backData")  # 设置本地数据缓存的路径，默认存在用户文件夹内

    # 在引擎中创建策略对象
    with open("CTA_setting.json") as f:
        d = json.load(f)[0]

    engine.initStrategy(StrategyBoll, d)
    
    # 开始跑回测
    engine.runBacktesting()
    
    # 显示回测结果
    engine.showDailyResult()
    engine.showBacktestingResult()

D:\ProgramData\Anaconda3\lib\site-packages\vnpy\trader\utils\htmlplot\core.py:8: UserWarning: Expected version of bokeh is 0.12.14, current version is 0.12.13
  warnings.warn("Expected version of bokeh is 0.12.14, current version is %s" % bokeh.__version__)


{'name': 'Strategy_Boll_arrayPerpear', 'className': 'StrategyBoll', 'symbolList': ['eth.usd.q:okef', 'btc.usd.q:okef'], 'posSize': 5, 'miniTick': 0.001, 'addTick': 10, 'maxPos': 1, 'BollBar': '15m', 'BollPeriod': 100, 'EMAPeriod': 4, 'pctPeriod': 50, 'divShield': 35, 'pctUp': 0.05, 'pctDown': 0.01, 'addPercent': 0.015, 'stopLoss': 0.02, 'timeStopLoss': 180, 'signalWait': 60, 'atrBar': '15m', 'atrPeriod': 30, 'atrShield': 40, 'timeframeMap': ['15m']}
初始仓位: {'eth.usd.q:okef_LONG': 0, 'eth.usd.q:okef_SHORT': 0, 'btc.usd.q:okef_LONG': 0, 'btc.usd.q:okef_SHORT': 0}
2019-05-17 10:20:05.878124	开始回测
2019-05-17 10:20:05.878124	策略初始化
2019-05-17 10:20:05.934765	准备载入数据：时间段:[20180605 06:00:00 , 20180610 06:00:00), 模式: bar
2019-05-17 10:20:06.738473	eth.usd.q:okef： 从本地缓存文件实取7197, 最大应取7200, 还需从数据库取3
2019-05-17 10:20:07.351756	btc.usd.q:okef： 从本地缓存文件实取7110, 最大应取7200, 还需从数据库取90
2019-05-17 10:20:07.552926	eth.usd.q:okef： 数据库也没能补到缺失的数据
2019-05-17 10:20:07.772652	btc.usd.q:okef： 数据库也没能补到缺失的数据
2019-05-17 1

KeyError: False

In [2]:
### 画图分析
candel=pd.DataFrame([bar.__dict__ for bar in engine.backtestData])
candel.head()
candel=candel[candel['vtSymbol']=='eth.usd.q:okef']
import os
trade_file = os.path.join(engine.logPath, "交割单.csv")
trades = htmlplot.read_transaction_file(trade_file)
mp = htmlplot.MultiPlot(os.path.join(engine.logPath, "transaction.html"))
# 设置主图， 传入蜡烛图和交割单
mp.set_main(candel, trades, "15m")
matlog = pd.DataFrame(engine.strategy.matlog).set_index('datetime')
mp.set_line(line=matlog[['divergence']])
mp.show()